In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chod 600 ~/.kaggle/kaggle.json

/bin/bash: line 1: chod: command not found


In [2]:
!pip install opendatasets

In [51]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jessicali9530/celeba-dataset")

Skipping, found downloaded files in "./celeba-dataset" (use force=True to force download)


In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import os
from PIL import Image
import torchvision
import numpy as np


In [53]:
class CelebAdataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform

        # Get all image file paths from the directory
        self.image_paths = [os.path.join(root_dir, img) for img in os.listdir(root_dir) if img.endswith('.jpg')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        # Apply the transform if provided
        if self.transform:
            image = self.transform(image)

        return image




In [54]:
# Define transformations (resize, crop, convert to tensor, normalize)

transform = transforms.Compose([
    transforms.Resize(64),       # Resize images to 64x64
    transforms.CenterCrop(64),    # Crop center to 64x64
    transforms.ToTensor(),         # Convert images to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize to [-1, 1]

])



In [55]:
# Load the CelebA dataset
dataset_path = '/content/celeba-dataset/img_align_celeba/img_align_celeba'
dataset = CelebAdataset(root_dir=dataset_path, transform=transform)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
# Check how many images are loaded
print(f"Number of images Loaded : {len(dataset)}")

Number of images Loaded : 202599


In [56]:
# Generator classes

class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, img_channels * 64 * 64),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 3, 64, 64)  # Reshape to image format
        return img


In [57]:
class Discriminator(nn.Module):
  def __init__(self, img_channels = 3):
    super(Discriminator, self).__init__()
    self.model=nn.Sequential(
        nn.Flatten(),
        nn.Linear(img_channels * 64 * 64, 1024),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(1024, 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(256, 1),
        nn.Sigmoid()
    )
  def forward(self, img):
    return self.model(img)

In [58]:
# Loss function and optimizers

adversarial_loss = nn.BCELoss()
generator = Generator(z_dim=100)
discriminator = Discriminator()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = generator.to(device)
discriminator = discriminator.to(device)

print(generator)
print(f" Discriminator parameter : {discriminator}")

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=1024, out_features=12288, bias=True)
    (7): Tanh()
  )
)
 Discriminator parameter : Discriminator(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=12288, out_features=1024, bias=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=512, out_features=256, bias=True)
    (6): LeakyReLU(negative_slope=0.2, inplace=True)
    (7): Linear(in_features=256, out_features=1, bias=True)
    (8): Sigmoid()
  )
)


In [ ]:
# Training loop

def train(generator, discriminator, dataloader, epochs=5):
  for epoch in range(epochs):
    for i, imgs in enumerate(dataloader):
      real_imgs = imgs.to(device)
      batch_size = real_imgs.size(0)
      valid = torch.ones(batch_size, 1).to(device)
      fake = torch.zeros(batch_size, 1).to(device)

      # Train Discriminator

      optimizer_D.zero_grad()
      real_loss = adversarial_loss(discriminator(real_imgs), valid)
      fake_loss = adversarial_loss(discriminator(generator(torch.randn(batch_size, 100).to(device)).detach()), fake)
      d_loss = (real_loss + fake_loss )/ 2
      d_loss.backward()
      optimizer_D.step()

      # Train the Generator

      optimizer_G.zero_grad()
      g_loss = adversarial_loss(discriminator(generator(torch.randn(batch_size, 100).to(device))), valid)
      g_loss.backward()
      optimizer_G.step()

      if i % 50 == 0:
       print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

   # Optionally, save generated images at each epoch
  save_generated_images(generator, epoch, device)



def save_generated_images(generator, epoch, device, num_images=16):
    z = torch.randn(num_images, 100).to(device)
    generated_imgs = generator(z).detach().cpu()
    grid = torchvision.utils.make_grid(generated_imgs, nrow=4, normalize=True)
    plt.imshow(np.transpose(grid, (1, 2, 0)))
    plt.title(f"Epoch {epoch}")
    plt.axis('off')
    plt.show()

# Start training
train(generator, discriminator, dataloader, epochs=10)


[Epoch 0/10] [Batch 0/1583] [D loss: 0.6919902563095093] [G loss: 0.7207863926887512]
[Epoch 0/10] [Batch 50/1583] [D loss: 0.2526938319206238] [G loss: 1.17840576171875]
[Epoch 0/10] [Batch 100/1583] [D loss: 0.16395944356918335] [G loss: 1.9114402532577515]
[Epoch 0/10] [Batch 150/1583] [D loss: 0.10349415242671967] [G loss: 2.0365304946899414]
[Epoch 0/10] [Batch 200/1583] [D loss: 0.3843155801296234] [G loss: 1.8570724725723267]
[Epoch 0/10] [Batch 250/1583] [D loss: 0.32253363728523254] [G loss: 0.9257451295852661]
[Epoch 0/10] [Batch 300/1583] [D loss: 0.07519432157278061] [G loss: 2.762653350830078]
[Epoch 0/10] [Batch 350/1583] [D loss: 0.1520053893327713] [G loss: 2.0360569953918457]
[Epoch 0/10] [Batch 400/1583] [D loss: 0.37260374426841736] [G loss: 4.871382713317871]
[Epoch 0/10] [Batch 450/1583] [D loss: 0.313314825296402] [G loss: 3.47910213470459]
[Epoch 0/10] [Batch 500/1583] [D loss: 1.2905088663101196] [G loss: 0.4242846965789795]
[Epoch 0/10] [Batch 550/1583] [D loss